### 📄 PDF 문서 변환 및 처리 초기화

이 노트북은 arXiv 논문에서 QA(질문-답변) 시스템을 구축하는 과정을 보여줍니다. 먼저 PDF 문서를 마크다운 형태로 변환하고 이미지를 추출하여 처리하는 작업부터 시작합니다.

#### 🔧 Docling 라이브러리를 사용한 PDF 변환

표준 모델에서 뮤온의 이상 자기 모멘트에 관한 물리학 논문을 대상으로, 고품질 이미지 추출과 수식 인식이 가능한 고급 PDF 파이프라인을 설정합니다.


In [2]:
import logging
import time
from pathlib import Path
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

# 로깅 설정 - INFO 레벨로 변환 과정을 모니터링
logging.basicConfig(level=logging.INFO)
_log = logging.getLogger(__name__)

# 입력 PDF 파일 경로와 출력 디렉토리 설정
input_doc_path = Path("data/The anomalous magnetic moment of the muon in the Standard Model.pdf")
output_dir = Path("standard_model")

# PDF 파이프라인 옵션 구성
pipeline_options = PdfPipelineOptions()
pipeline_options.images_scale = 2.0  # 이미지 해상도 스케일 설정
pipeline_options.generate_picture_images = True         # 그림 이미지 생성 활성화
pipeline_options.do_formula_enrichment = True
pipeline_options.do_picture_classification = True

# 문서 변환기 초기화 - PDF 형식 옵션과 함께 설정
doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

# 변환 시작 시간 기록
start_time = time.time()

# PDF 문서 변환 실행
conv_res = doc_converter.convert(input_doc_path)

INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash 30758e3e50e33b1c127abb388dad2ec7
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered picture descriptions: ['vlm', 'api']
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.pipeline.base_pipeline:Processing document The anomalous magnetic moment of the muon in the Standard Model.pdf
c:\U

#### 🔍 문서 구조 분석

변환된 문서에서 어떤 유형의 요소들이 발견되는지 확인합니다. 이를 통해 텍스트, 이미지, 표, 수식 등의 분포를 파악할 수 있습니다.


In [3]:
# 문서에서 발견되는 고유한 element 타입들을 확인
element_types = set()
for element, _level in conv_res.document.iterate_items():
    element_types.add(type(element).__name__)

print("문서에서 발견된 고유한 element 타입들:")
for element_type in sorted(element_types):
    print(f"- {element_type}")


문서에서 발견된 고유한 element 타입들:
- FormulaItem
- ListItem
- PictureItem
- SectionHeaderItem
- TableItem
- TextItem


#### 💾 마크다운 파일 저장 및 완료

변환된 문서를 이미지 참조가 포함된 마크다운 형식으로 저장합니다. 이미지들은 별도 디렉토리에 저장되며, 마크다운에서는 상대 경로로 참조됩니다.


In [4]:
doc_filename = "physics_muon_paper"

# 출력 디렉토리가 존재하지 않으면 생성
output_dir.mkdir(parents=True, exist_ok=True)

# 이미지 참조가 포함된 마크다운 파일 저장 
md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)

# 변환 완료 시간 계산
end_time = time.time() - start_time

# 변환 완료 로그 출력
_log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")

INFO:__main__:Document converted and figures exported in 1107.61 seconds.


### 🖼️ 이미지 캡션 생성 및 VLM 활용

논문의 이미지들을 텍스트로 변환하기 위해 Vision Language Model(VLM)을 사용합니다. 

#### 🧪 개별 이미지 테스트

Gemini 2.5 Flash 모델을 사용하여 단일 이미지에 대한 캡션 생성을 테스트해봅니다. 이미지를 base64로 인코딩하여 API에 전송합니다.


In [5]:
import base64
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20")

# 로컬 이미지 파일 경로
image_path = "standard_model\physics_muon_paper-with-image-refs_artifacts\image_000113_5d0f0f98dd32385cc160b1f470c6fe41e41cd81e6795a6af111ac6f2d451f1a0.png"

# 로컬 이미지 파일을 base64로 인코딩
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

message_local = HumanMessage(
    content=[
        {"type": "text", "text": "이 이미지를 텍스트로 대체하고자 합니다. 해당 이미지에 대한 설명을 한글로 생성해주세요."},
        {"type": "image_url", "image_url": f"data:image/png;base64,{encoded_image}"},
    ]
)
result_local = llm.invoke([message_local])
print(f"Response for local image: {result_local.content}")

Response for local image: 이 이미지는 두 개의 나란히 배치된 그래프로, η (에타) 및 η' (에타 프라임) 중간자의 Q² 의존적인 전이 형태 인자(form factor)를 보여줍니다. 두 그래프 모두 가로축은 Q² (운동량 전달 제곱)를 GeV² 단위로, 세로축은 형태 인자 값을 GeV 단위로 표시합니다. 그래프는 여러 이론적 계산 결과("This work", "DSE", "CA")와 실험 데이터("CLEO", "CELLO", "L3")를 비교합니다.

**왼쪽 그래프: Q² Fηγ*γ*(-Q², 0)**
*   **제목:** η 중간자의 전이 형태 인자 Q² Fηγ*γ*(-Q², 0)를 나타냅니다.
*   **축 범위:** 가로축 Q²는 0에서 2 GeV²까지, 세로축은 0에서 0.2 GeV까지 범위입니다.
*   **범례:**
    *   **This work:** 연한 파란색 음영 영역과 그 중앙의 파란색 실선으로 표시된 현재 연구의 이론적 예측입니다.
    *   **CLEO:** 녹색 원형 점으로 표시된 실험 데이터입니다.
    *   **CELLO:** 파란색 사각형 점으로 표시된 실험 데이터입니다.
    *   **DSE:** 녹색 음영 영역으로 표시된 이론적 모델(Dyson-Schwinger Equations)입니다.
    *   **CA:** 주황색 음영 영역으로 표시된 이론적 모델(Chiral Anomaly)입니다.
*   **데이터 경향:**
    *   모든 곡선과 데이터는 Q²가 증가함에 따라 0에서 시작하여 점진적으로 증가하는 경향을 보입니다.
    *   "This work"의 파란색 예측 밴드는 "CELLO" 실험 데이터 포인트와 잘 일치하는 것으로 보입니다.
    *   "CLEO" 실험 데이터 포인트는 "DSE" 및 "CA" 이론적 예측 밴드와 더 가깝게 위치합니다.
    *   이론적 예측의 음영 영역은 불확실성을 나타내며, Q²가 커질수록 이 불확실성 영역도 넓어집니다.
    *   

#### ⚡ 배치 처리를 통한 대량 이미지 캡션 생성

논문에 포함된 모든 이미지(124개)에 대해 자동으로 캡션을 생성하는 고급 시스템입니다.

주요 기능:
- 이미지 주변 텍스트 컨텍스트 추출 및 활용
- Rate limit을 준수하는 배치 처리
- 실패한 이미지에 대한 개별 재처리
- 생성된 캡션으로 이미지 참조 자동 대체


In [ ]:
# 마크다운 파일에서 이미지 참조를 찾아서 VLM으로 캡션 생성 후 대체하는 코드

import re
import time
import asyncio
from pathlib import Path
from typing import List, Tuple

def extract_surrounding_context(content, image_ref, context_lines=3):
    """
    이미지 참조 주변의 텍스트 컨텍스트를 추출
    """
    lines = content.split('\n')
    
    # 이미지 참조가 있는 라인 찾기
    image_line_idx = None
    for i, line in enumerate(lines):
        if image_ref in line:
            image_line_idx = i
            break
    
    if image_line_idx is None:
        return ""
    
    # 앞뒤 context_lines만큼의 라인 추출
    start_idx = max(0, image_line_idx - context_lines)
    end_idx = min(len(lines), image_line_idx + context_lines + 1)
    
    context_lines_list = lines[start_idx:end_idx]
    
    # 이미지 참조 라인 제외하고 컨텍스트만 추출
    context_lines_list = [line for line in context_lines_list if image_ref not in line]
    
    # 빈 라인과 마크다운 헤더 등 정리
    context_text = '\n'.join(context_lines_list).strip()
    
    return context_text

def create_caption_message(image_path: Path, context: str) -> HumanMessage:
    """
    VLM 캡션 생성을 위한 메시지 생성
    """
    try:
        # 이미지를 base64로 인코딩
        with open(image_path, "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
        
        # 컨텍스트를 포함한 프롬프트 생성
        prompt = f"""다음은 학술 논문의 일부입니다. 이미지 주변의 텍스트 컨텍스트를 참고하여 이미지를 설명해주세요.

주변 텍스트:
{context}

위 텍스트와 관련된 이미지를 보고 적절한 캡션을 한글로 생성해주세요. 
캡션은 간결하고 정확하며, 주변 텍스트의 맥락에 맞게 작성해주세요."""

        message = HumanMessage(
            content=[
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": f"data:image/png;base64,{encoded_image}"},
            ]
        )
        
        return message
        
    except Exception as e:
        print(f"메시지 생성 중 오류 발생: {e}")
        return None

def process_images_with_lcel_batch(image_data_list: List[Tuple[str, str, Path, str]], batch_size: int = 5):
    """
    LCEL batch를 사용하여 이미지들을 배치로 처리
    Gemini 2.5 Flash Preview 05-20: 10 RPM, 250,000 TPM, 500 RPD
    """
    captions = {}
    total_batches = (len(image_data_list) + batch_size - 1) // batch_size
    
    print(f"총 {len(image_data_list)}개 이미지를 {total_batches}개 배치로 처리합니다.")
    
    for batch_idx in range(0, len(image_data_list), batch_size):
        batch = image_data_list[batch_idx:batch_idx + batch_size]
        batch_num = batch_idx // batch_size + 1
        
        print(f"배치 {batch_num}/{total_batches} 처리 중... ({len(batch)}개 이미지)")
        
        # 배치용 메시지 리스트 생성
        batch_messages = []
        batch_refs = []
        
        for alt_text, image_path, full_image_path, context in batch:
            print(f"  - 메시지 준비 중: {image_path}")
            message = create_caption_message(full_image_path, context)
            if message:
                batch_messages.append([message])  # LCEL batch는 각 입력이 리스트여야 함
                batch_refs.append(f"![{alt_text}]({image_path})")
            else:
                print(f"  - 메시지 생성 실패: {image_path}")
                batch_refs.append(f"![{alt_text}]({image_path})")
                captions[f"![{alt_text}]({image_path})"] = f"[이미지 캡션 생성 실패: {image_path}]"
        
        # LCEL batch 처리
        if batch_messages:
            try:
                print(f"  - LCEL batch 실행 중... ({len(batch_messages)}개 메시지)")
                batch_results = llm.batch(batch_messages)
                
                # 결과 처리
                for i, result in enumerate(batch_results):
                    if i < len(batch_refs):
                        caption = result.content.strip() if hasattr(result, 'content') else str(result).strip()
                        captions[batch_refs[i]] = caption
                        print(f"  - 캡션 생성 완료: {batch_refs[i][:50]}...")
                
            except Exception as e:
                print(f"  - 배치 처리 중 오류 발생: {e}")
                # 오류 발생 시 개별 처리로 폴백
                for i, message_list in enumerate(batch_messages):
                    try:
                        result = llm.invoke(message_list)
                        caption = result.content.strip()
                        captions[batch_refs[i]] = caption
                        print(f"  - 개별 처리 완료: {batch_refs[i][:50]}...")
                        time.sleep(6.5)  # Rate limit 준수
                    except Exception as individual_error:
                        print(f"  - 개별 처리 실패: {individual_error}")
                        captions[batch_refs[i]] = f"[이미지 캡션 생성 실패]"
        
        # 배치 간 Rate limit 준수를 위한 대기
        # RPM 10 제한: 배치 크기가 5라면 30초 대기 (6초 * 5)
        if batch_num < total_batches:
            wait_time = batch_size * 6.5
            print(f"배치 {batch_num} 완료. Rate limit 준수를 위해 {wait_time}초 대기...")
            time.sleep(wait_time)
    
    return captions

def replace_image_refs_with_captions(md_file_path, output_dir, doc_filename):
    """
    마크다운 파일의 이미지 참조를 VLM 생성 캡션으로 대체하여 새로운 파일로 저장
    """
    # 마크다운 파일 읽기
    with open(md_file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 이미지 참조 패턴 찾기 (![alt_text](image_path) 형식)
    image_pattern = r'!\[([^\]]*)\]\(([^)]+)\)'
    matches = re.findall(image_pattern, content)
    
    print(f"발견된 이미지 참조 개수: {len(matches)}")
    
    # 처리할 이미지 데이터 준비
    image_data_list = []
    
    for alt_text, image_path in matches:
        # 상대 경로를 절대 경로로 변환
        if not Path(image_path).is_absolute():
            full_image_path = output_dir / image_path
        else:
            full_image_path = Path(image_path)
        
        if full_image_path.exists():
            # 이미지 참조 주변의 컨텍스트 추출
            image_ref = f"![{alt_text}]({image_path})"
            surrounding_context = extract_surrounding_context(content, image_ref, context_lines=3)
            
            image_data_list.append((alt_text, image_path, full_image_path, surrounding_context))
            print(f"이미지 추가: {image_path}")
            print(f"컨텍스트: {surrounding_context[:100]}{'...' if len(surrounding_context) > 100 else ''}\n")
        else:
            print(f"이미지 파일을 찾을 수 없음: {full_image_path}")
    
    if not image_data_list:
        print("처리할 이미지가 없습니다.")
        return md_file_path
    
    # LCEL batch 처리로 캡션 생성
    print("LCEL batch를 사용한 캡션 생성을 시작합니다...")
    start_time = time.time()
    
    captions = process_images_with_lcel_batch(image_data_list, batch_size=5)
    
    # 이미지 참조를 캡션으로 대체
    for image_ref, caption in captions.items():
        content = content.replace(image_ref, f"[이미지 캡션: {caption}]")
        print(f"대체 완료: {image_ref[:50]}... -> {caption[:50]}...")
    
    # 캡션이 적용된 새로운 마크다운 파일 저장
    captioned_md_filename = output_dir / f"{doc_filename}-with-captions.md"
    with open(captioned_md_filename, 'w', encoding='utf-8') as f:
        f.write(content)
    
    end_time = time.time() - start_time
    print(f"캡션 생성 완료! 총 소요 시간: {end_time:.2f}초")
    print(f"총 {len(captions)}개 이미지 캡션 생성됨")
    print(f"캡션이 적용된 새로운 파일 저장: {captioned_md_filename}")
    
    return captioned_md_filename

# 이미지 참조를 캡션으로 대체하여 새로운 파일로 저장
captioned_file = replace_image_refs_with_captions(md_filename, output_dir, doc_filename)
# End of Selection

### 📚 문서 분할 및 임베딩 준비

#### ✂️ 텍스트 청킹

캡션이 생성된 마크다운 문서를 RAG 시스템에 적합한 크기의 청크로 분할합니다. RecursiveCharacterTextSplitter를 사용하여 의미 있는 구조를 유지하면서 분할합니다.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["##\n","\n\n", "\n", " ", ""]
)

# 캡션이 적용된 파일 내용 읽기
with open(captioned_file, 'r', encoding='utf-8') as f:
    captioned_content = f.read()

# Document 객체로 생성
document = Document(page_content=captioned_content, metadata={"source": str(captioned_file)})

# split_documents를 사용하여 Document 객체들로 분할
splits = text_splitter.split_documents([document])

# splits 길이 분포 파악
split_lengths = [len(split.page_content) for split in splits]
print(f"총 분할된 청크 수: {len(splits)}")
print(f"평균 청크 길이: {sum(split_lengths) / len(split_lengths):.1f} 문자")
print(f"최소 청크 길이: {min(split_lengths)} 문자")
print(f"최대 청크 길이: {max(split_lengths)} 문자")

#### 👀 분할 결과 확인

분할된 첫 번째 청크의 내용을 확인하여 청킹이 올바르게 수행되었는지 검토합니다.


In [ ]:
splits[0]

### 🗄️ 벡터 데이터베이스 설정 및 하이브리드 검색

#### 💾 Qdrant 벡터 스토어 구축

Dense embedding과 Sparse embedding을 결합한 하이브리드 검색 시스템을 구축합니다:
- **Dense embedding**: BGE-M3 모델을 사용한 의미적 유사도 검색
- **Sparse embedding**: BM25 기반 키워드 검색
- **하이브리드 검색**: 두 방식을 결합하여 더 정확한 검색 성능 확보


In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams

# Qdrant 클라이언트 설정
client = QdrantClient(host="localhost", port=6333)

dense_embeddings = OllamaEmbeddings(model="bge-m3")
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

# 컬렉션 이름 설정
collection_name = f"arxiv_{doc_filename}"
# try:
    # client.create_collection(
    #     collection_name=collection_name,
    #     vectors_config={"dense": VectorParams(size=1024, distance=Distance.COSINE)},
    #     sparse_vectors_config={
    #         "sparse": SparseVectorParams(index=models.SparseIndexParams(on_disk=False))
    #     },
    # )
# print(f"새 컬렉션 '{collection_name}' 생성됨")
    # 문서를 벡터 스토어에 추가
# print("문서를 벡터 스토어에 추가 중...")
qdrant = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=dense_embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse",
)
    # qdrant.add_documents(splits)
    # print(f"✅ {len(splits)}개의 문서가 Qdrant에 저장되었습니다.")

# except:
#     qdrant = QdrantVectorStore(
#         client=client,
#         collection_name=collection_name,
#         embedding=dense_embeddings,
#         sparse_embedding=sparse_embeddings,
#         retrieval_mode=RetrievalMode.HYBRID,
#         vector_name="dense",
#         sparse_vector_name="sparse",
#     )
#     print(f"✅ {len(splits)}개의 문서가 Qdrant에 저장되었습니다.")

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/arxiv_physics_muon_paper "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/arxiv_physics_muon_paper "HTTP/1.1 200 OK"


새 컬렉션 'arxiv_physics_muon_paper' 생성됨
문서를 벡터 스토어에 추가 중...


#### 🔍 리트리버 초기화

벡터 스토어를 기반으로 한 리트리버를 설정합니다. 상위 10개 문서를 검색하도록 구성합니다.


In [8]:
retriever = qdrant.as_retriever(
    search_kwargs={"k": 10}
)

#### 🎨 검색 결과 출력 함수

검색된 문서들을 깔끔하게 포맷팅하여 출력하는 유틸리티 함수입니다. 문서의 메타데이터와 내용을 구조화된 형태로 보여줍니다.


In [9]:
def print_search_results(docs, title):
    """검색 결과를 이쁘게 출력하는 함수"""
    print(f"{title}")
    print("=" * 80)
    print(f"검색된 문서 수: {len(docs)}개\n")
    
    for i, doc in enumerate(docs, 1):
        print(f"📄 문서 {i}")
        print(f"📂 출처: {doc.metadata.get('source', 'N/A')}")
        print(f"📃 페이지: {doc.metadata.get(' page', 'N/A')}")
        print(f"📝 내용 미리보기:")
        print(doc.page_content)
        print("-" * 40)


#### 🧪 기본 검색 테스트

HLbL(Hadronic Light-by-Light) 불확실성에 관한 질문으로 검색 시스템을 테스트합니다. 이는 뮤온 g-2 실험의 핵심 주제 중 하나입니다.


In [10]:
# 검색 쿼리
query = "HLbL 불확실성이 2020 WP 대비 얼마나 줄었는가?"

# 기본 retriever 검색
basic_docs = retriever.invoke(query)
print_search_results(basic_docs, "🔍 기본 Retriever 검색 결과:")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/arxiv_physics_muon_paper/points/query "HTTP/1.1 200 OK"


🔍 기본 Retriever 검색 결과:
검색된 문서 수: 10개

📄 문서 1
📂 출처: standard_model\physics_muon_paper-with-captions.md
📃 페이지: N/A
📝 내용 미리보기:
Table 33: Comparison of the key results from this work (WP25), as given in Table 1, to the corresponding numbers from WP20 [1] (in units of 10 -11 ). Note that the 'HLbL (lattice)' result from WP20 has been adapted to include the charm-loop contribution. The entry 'HVP (LO + NLO + NNLO)' derives from HVP LO (lattice) [WP25] and HVP LO ( e + e -) [WP20], respectively. The asterisk indicates that the LO HVP value from WP20 was based on e + e -data only, while in Table 5 we also include the current status for τ -based evaluations.

## 9. Conclusions and outlook

In this second edition of the White Paper on the muon g -2, we have charted the progress that has been achieved since 2020 in evaluating the contributions from the electromagnetic (QED), electroweak (EW), and strong (QCD) interactions to a µ .
----------------------------------------
📄 문서 2
📂 출처: standard_mode

### 🤖 LangGraph를 활용한 RAG 시스템 구축

#### ⚙️ RAG 워크플로우 설정

LangGraph를 사용하여 체계적인 RAG(Retrieval-Augmented Generation) 파이프라인을 구축합니다:

1. **State 정의**: 질문, 검색된 문서, 생성된 답변을 관리
2. **LLM 설정**: Gemma3 4B 모델을 사용한 답변 생성
3. **프롬프트 템플릿**: 한국어 답변을 위한 최적화된 프롬프트
4. **워크플로우 노드**: 문서 검색과 답변 생성의 단계별 처리


In [11]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_core.documents import Document
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# State 정의
class RAGState(TypedDict):
    question: str
    documents: List[Document]
    answer: str

# LLM 초기화
llm = ChatOllama(
    model="gemma3:4b",
    temperature=0
)

# RAG 프롬프트 템플릿
rag_prompt = ChatPromptTemplate.from_template("""
다음 문서들을 참고하여 질문에 답변해주세요.

문서들:
{context}

질문: {question}

답변은 한국어로 작성하고, 문서의 내용을 바탕으로 정확하고 자세하게 답변해주세요.
답변:
""")

def retrieve_documents(state: RAGState) -> RAGState:
    """문서 검색 단계"""
    print("📚 문서 검색 중...")
    question = state["question"]
    documents = retriever.invoke(question)
    print(f"✅ {len(documents)}개의 문서를 검색했습니다.")
    
    return {
        "question": question,
        "documents": documents,
        "answer": ""
    }

def generate_answer(state: RAGState) -> RAGState:
    """답변 생성 단계"""
    print("🤖 답변 생성 중...")
    question = state["question"]
    documents = state["documents"]
    
    # 문서 내용을 컨텍스트로 결합
    context = "\n\n".join([doc.page_content for doc in documents])
    
    # LLM 체인 구성
    chain = rag_prompt | llm | StrOutputParser()
    
    # 답변 생성
    answer = chain.invoke({
        "context": context,
        "question": question
    })
    
    print("✅ 답변이 생성되었습니다.")
    
    return {
        "question": question,
        "documents": documents,
        "answer": answer
    }

# LangGraph 워크플로우 구성
workflow = StateGraph(RAGState)

# 노드 추가
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("generate", generate_answer)

# 엣지 추가
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

# 그래프 컴파일
rag_app = workflow.compile()

#### 🚀 완전한 RAG 시스템 실행

구축된 RAG 시스템을 사용하여 실제 질문에 대한 답변을 생성합니다. 
- 문서 검색과 답변 생성 과정을 실시간으로 모니터링
- 스트리밍 모드로 답변 생성 과정을 단계별로 확인


In [12]:
question = "HLbL 불확실성이 2020 WP 대비 얼마나 줄었는가?"
initial_state = {"question": question, "documents": [], "answer": ""}
result=[]
for chunk, metadata in rag_app.stream(initial_state, stream_mode="messages"):
    if chunk.content:
        result.append(chunk)
        print(chunk.content, end="", flush=True)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/arxiv_physics_muon_paper/points/query "HTTP/1.1 200 OK"


📚 문서 검색 중...
✅ 10개의 문서를 검색했습니다.
🤖 답변 생성 중...


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


문서에 따르면, HLbL의 불확실성이 2020 WP(WP20) 대비 약 두 배 감소했습니다. 2020 WP20의 결과는 92(19) × 10 -11 였지만, 새로운 측정 결과들이 발표되면서 현재의 세계 평균값은 이보다 훨씬 더 정밀해졌습니다. 특히, 2021년에 발표된 Fermilab g-2 실험 결과(460 ppb)와 2023년에 발표된 결과(200 ppb)는 이전 결과보다 훨씬 더 높은 정밀도를 제공하며, 이로 인해 HLbL의 불확실성이 크게 개선되었습니다. 또한, J-PARC 실험도 준비 중이며, 이 실험 결과 또한 최종적으로 세계 평균값의 신뢰도를 높일 것으로 기대됩니다.
✅ 답변이 생성되었습니다.


### 🎯 RAG 시스템 완성

이 노트북을 통해 다음과 같은 완전한 RAG 시스템을 구축했습니다:

1. **📄 PDF 처리**: Docling을 통한 고품질 문서 변환
2. **🖼️ 이미지 처리**: VLM을 활용한 자동 캡션 생성
3. **📚 문서 분할**: 의미 있는 청크 단위로 분할
4. **🗄️ 벡터 DB**: 하이브리드 검색이 가능한 Qdrant 설정
5. **🤖 QA 시스템**: LangGraph 기반 체계적 답변 생성

이제 arXiv 논문에 대한 정확하고 상세한 질문-답변이 가능합니다!
